In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import scanpy as sc
import anndata
from scipy import io
from scipy.sparse import coo_matrix, csr_matrix
import numpy as np
import os
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import scirpy
import h5py
import anndata

In [ ]:
from matplotlib.pyplot import rc_context

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
sc.set_figure_params(scanpy=True, fontsize=12)

In [ ]:
cd4=mem[mem.obs['celltype'].isin(['CD4 Tmem'])]

In [ ]:
cd4 = cd4.raw.to_adata()

In [ ]:
cd4.uns['log1p']['base'] = None

In [ ]:
sc.pp.highly_variable_genes(cd4, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
cd4.raw = cd4

In [ ]:
cd4.layers["counts"] = cd4.X.copy()

In [ ]:
cd4 = cd4[:, cd4.var.highly_variable]

In [ ]:
import hotspot
#import muon as mu
import mplscience
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns
#import mplscience

from scipy.io import mmread
from scipy.sparse import csr_matrix

In [ ]:
hs = hotspot.Hotspot(cd4, model='normal',latent_obsm_key='X_umap',layer_key="counts")

hs.create_knn_graph(
    weighted_graph=False, n_neighbors=30,
)

In [ ]:
hs_results = hs.compute_autocorrelations(jobs=16)

In [ ]:
hs_genes = hs_results.loc[hs_results.FDR < 0.05].sort_values('Z', ascending=False).head(500).index

In [ ]:
hs_genes2=hs_genes[hs_genes.str.contains('RPL')| hs_genes.str.contains('RPS')|hs_genes.str.contains('MT-')|hs_genes.str.contains('\.')|hs_genes.str.contains('-AS')
                  |hs_genes.str.contains('-DT') |hs_genes.str.contains('-IT1')|hs_genes.str.contains('LINC')
                  |hs_genes.str.contains('orf')]

In [ ]:
hs_genes3=hs_genes.drop(hs_genes2)

In [ ]:
lcz = hs.compute_local_correlations(hs_genes3, jobs=16)

In [ ]:
modules = hs.create_modules(
    min_gene_threshold=20, core_only=True, fdr_threshold=0.05
)

In [ ]:
hs.plot_local_correlations()
#plt.savefig('cd4module.pdf',bbox_inches='tight')

In [ ]:
module_scores = hs.calculate_module_scores()

In [ ]:
module_cols = []
for c in module_scores.columns:
    key = f"Module {c}"
    cd4.obs[key] = module_scores[c]
    module_cols.append(key)

In [ ]:
sc.pl.umap(cd4, color=module_cols,size=10)

In [ ]:
sc.pl.dotplot(cd4, module_cols, groupby=['Age_combine'])